# Simple Unet CV Training

1. Basic Unet model with resnet blocks.
2. starting layer 128*128, 16 channels
3. Middle layer 4*4, 512 channels

In [ ]:
import numpy as np
import pandas as pd
import glob
from random import randint
from PIL import Image

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

import random

from sklearn.model_selection import train_test_split,StratifiedKFold
from skimage.transform import resize

from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model, save_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
from keras.losses import binary_crossentropy
from tqdm import tqdm_notebook
from keras import backend as K

# Params and helpers

In [ ]:
img_size_ori = 101
img_size_target = 128

def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    #res = np.zeros((img_size_target, img_size_target), dtype=img.dtype)
    #res[:img_size_ori, :img_size_ori] = img
    #return res
    
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)
    #return img[:img_size_ori, :img_size_ori]

In [ ]:
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)


In [ ]:
# close approximation of competition metric

import tensorflow as tf
#Score the model and do a threshold optimization by the best IoU.

# src: https://www.kaggle.com/aglotero/another-iou-metric
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in


    true_objects = 2
    pred_objects = 2

    # Jiaxin fin that if all zeros, then, the background is treated as object
    temp1 = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=([0,0.5,1], [0,0.5, 1]))
#     temp1 = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))
    #print(temp1)
    intersection = temp1[0]
    #print("temp2 = ",temp1[1])
    #print(intersection.shape)
   # print(intersection)
    # Compute areas (needed for finding the union between all objects)
    #print(np.histogram(labels, bins = true_objects))
    area_true = np.histogram(labels,bins=[0,0.5,1])[0]
    #print("area_true = ",area_true)
    area_pred = np.histogram(y_pred, bins=[0,0.5,1])[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection
    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    intersection[intersection == 0] = 1e-9
    
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union

    # Precision helper function
    def precision_at(threshold, iou):
        matches = iou > threshold
        true_positives = np.sum(matches, axis=1) == 1   # Correct objects
        false_positives = np.sum(matches, axis=0) == 0  # Missed objects
        false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

def iou_metric_batch(y_true_in, y_pred_in):
    y_pred_in = y_pred_in > 0.5 # added by sgx 20180728
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    #print("metric = ",metric)
    return np.mean(metric)

def my_iou_metric(label, pred):
    metric_value = tf.py_func(iou_metric_batch, [label, pred], tf.float64)
    return metric_value

In [ ]:
train_df = pd.read_csv("../input/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../input/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

In [ ]:
train_df["images"] = [np.array(load_img("../input/train/images/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

In [ ]:
train_df["masks"] = [np.array(load_img("../input/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

# Calculating the salt coverage and salt coverage classes
Counting the number of salt pixels in the masks and dividing them by the image size. Also create 11 coverage classes, -0.1 having no salt at all to 1.0 being salt only.
Plotting the distribution of coverages and coverage classes, and the class against the raw coverage.

In [ ]:
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size_ori, 2)

In [ ]:
def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,5))
sns.distplot(train_df.coverage, kde=False, ax=axs[0])
sns.distplot(train_df.coverage_class, bins=10, kde=False, ax=axs[1])
plt.suptitle("Salt coverage")
axs[0].set_xlabel("Coverage")
axs[1].set_xlabel("Coverage class")

In [ ]:
plt.scatter(train_df.coverage, train_df.coverage_class)
plt.xlabel("Coverage")
plt.ylabel("Coverage class")

# Plotting the depth distributions
Separatelty plotting the depth distributions for the training and the testing data.

In [ ]:
sns.distplot(train_df.z, label="Train")
sns.distplot(test_df.z, label="Test")
plt.legend()
plt.title("Depth distribution")

# Show some example images

In [ ]:
max_images = 60
grid_width = 15
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
for i, idx in enumerate(train_df.index[:max_images]):
    img = train_df.loc[idx].images
    mask = train_df.loc[idx].masks
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(img, cmap="Greys")
    ax.imshow(mask, alpha=0.3, cmap="Greens")
    ax.text(1, img_size_ori-1, train_df.loc[idx].z, color="black")
    ax.text(img_size_ori - 1, 1, round(train_df.loc[idx].coverage, 2), color="black", ha="right", va="top")
    ax.text(1, 1, train_df.loc[idx].coverage_class, color="black", ha="left", va="top")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.suptitle("Green: salt. Top-left: coverage class, top-right: salt coverage, bottom-left: depth")

# Create train/validation split stratified by salt coverage

In [ ]:
# 5 fold cross validation , stratified based on salt coverage

ids_train = []
ids_valid = []
skf = StratifiedKFold(n_splits=5, random_state=1338)
i = 0
for indices in skf.split(X=train_df.index.values,y=train_df.coverage_class):
    if i==4:
        ids_train = indices[0]
        ids_valid = indices[1]
        break
    i += 1

x_train = np.array(train_df.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)[ids_train]
x_valid = np.array(train_df.images.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)[ids_valid]
y_train = np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)[ids_train]
y_valid = np.array(train_df.masks.map(upsample).tolist()).reshape(-1, img_size_target, img_size_target, 1)[ids_valid]

In [ ]:
print(x_train.shape, x_valid.shape)
print(y_train.shape, y_valid.shape)

# Build model

In [ ]:
def iou_coef(y_true, y_pred, smooth=1):
    print(y_true.shape, y_pred.shape)
    iou = 0.
    for i in range(16):
        intersection = K.sum(K.abs(y_true[i,:,:,:] * y_pred[i,:,:,:]))
        print(intersection)
        union = K.sum(y_true[i,:,:,:]) + K.sum(y_pred[i,:,:,:]) - intersection
        iou = (intersection + smooth) / ( union + smooth)
    return iou/16

def iou_coef_loss(y_true, y_pred):
    return -iou_coef(y_true, y_pred)

In [ ]:
from keras.optimizers import Adam
from keras.layers import Add
# Build U-Net model
input_img = Input((img_size_target, img_size_target, 1), name='img')

c1 = Conv2D(16, (3, 3), activation='relu', padding='same') (input_img)
c1 = BatchNormalization()(c1)
c1 = Conv2D(16, (3, 3), activation='relu', padding='same') (c1)
c1 = BatchNormalization()(c1)
d1 = Conv2D(16, (1, 1), activation='relu', padding='same') (input_img)
c1 = Add()([c1, d1])
c1 = BatchNormalization()(c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='relu', padding='same') (p1)
c2 = BatchNormalization()(c2)
c2 = Conv2D(32, (3, 3), activation='relu', padding='same') (c2)
c2 = BatchNormalization()(c2)
d2 = Conv2D(32, (1, 1), activation='relu', padding='same') (p1)
c2 = Add()([c2, d2])
c2 = BatchNormalization()(c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='relu', padding='same') (p2)
c3 = BatchNormalization()(c3)
c3 = Conv2D(64, (3, 3), activation='relu', padding='same') (c3)
c3 = BatchNormalization()(c3)
d3 = Conv2D(64, (1, 1), activation='relu', padding='same') (p2)
c3 = Add()([c3, d3])
c3 = BatchNormalization()(c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='relu', padding='same') (p3)
c4 = BatchNormalization()(c4)
c4 = Conv2D(128, (3, 3), activation='relu', padding='same') (c4)
c4 = BatchNormalization()(c4)
d4 = Conv2D(128, (1, 1), activation='relu', padding='same') (p3)
c4 = Add()([c4, d4])
c4 = BatchNormalization()(c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
#p4 = Dropout(0.5)(p4)

c5 = Conv2D(256, (3, 3), activation='relu', padding='same') (p4)
c5 = BatchNormalization()(c5)
c5 = Conv2D(256, (3, 3), activation='relu', padding='same') (c5)
c5 = BatchNormalization()(c5)
d5 = Conv2D(256, (1, 1), activation='relu', padding='same') (p4)
c5 = Add()([c5, d5])
c5 = BatchNormalization()(c5)
p5 = MaxPooling2D(pool_size=(2, 2)) (c5)

c6 = Conv2D(512, (3, 3), activation='relu', padding='same') (p5)
c6 = BatchNormalization()(c6)
c6 = Dropout(0.5)(c6)
c6 = Conv2D(512, (3, 3), activation='relu', padding='same') (c6)
c6 = BatchNormalization()(c6)

u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c5])
c7 = Conv2D(256, (3, 3), activation='relu', padding='same') (u7)
c7 = BatchNormalization()(c7)
c7 = Conv2D(256, (3, 3), activation='relu', padding='same') (c7)
c7 = BatchNormalization()(c7)
d7 = Conv2D(256, (1, 1), activation='relu', padding='same') (u7)
c7 = Add()([c7, d7])
c7 = BatchNormalization()(c7)

u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c4])
#u7 = Dropout(0.5)(u7)
c8 = Conv2D(128, (3, 3), activation='relu', padding='same') (u8)
c8 = BatchNormalization()(c8)
c8 = Conv2D(128, (3, 3), activation='relu', padding='same') (c8)
c8 = BatchNormalization()(c8)
d8 = Conv2D(128, (1, 1), activation='relu', padding='same') (u8)
c8 = Add()([c8, d8])
c8 = BatchNormalization()(c8)

u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c3])
#u8 = Dropout(0.5)(u8)
c9 = Conv2D(64, (3, 3), activation='relu', padding='same') (u9)
c9 = BatchNormalization()(c9)
c9 = Conv2D(64, (3, 3), activation='relu', padding='same') (c9)
c9 = BatchNormalization()(c9)
d9 = Conv2D(64, (1, 1), activation='relu', padding='same') (u9)
c9 = Add()([c9, d9])
c9 = BatchNormalization()(c9)

u10 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c9)
u10 = concatenate([u10, c2])
#u8 = Dropout(0.5)(u8)
c10 = Conv2D(32, (3, 3), activation='relu', padding='same') (u10)
c10 = BatchNormalization()(c10)
c10 = Conv2D(32, (3, 3), activation='relu', padding='same') (c10)
c10 = BatchNormalization()(c10)
d10 = Conv2D(32, (1, 1), activation='relu', padding='same') (u10)
c10 = Add()([c10, d10])
c10 = BatchNormalization()(c10)

u11 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c10)
u11 = concatenate([u11, c1], axis=3)
#u9 = Dropout(0.6)(u9)
c11 = Conv2D(16, (3, 3), activation='relu', padding='same') (u11)
c11 = BatchNormalization()(c11)
c11 = Conv2D(16, (2, 2), activation='relu', padding='same') (c11)
c11 = BatchNormalization()(c11)
d11 = Conv2D(16, (1, 1), activation='relu', padding='same') (u11)
c11 = Add()([c11, d11])
c11 = BatchNormalization()(c11)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c11)
opt = Adam(lr=0.0012)
model = Model(inputs=[input_img], outputs=[outputs])
model.compile(optimizer=opt, loss=bce_dice_loss, metrics=[my_iou_metric]) 
model.summary()

# Data augmentation

1. Horizontal flips
2. corner based random crops

In [ ]:
from random import randint, sample
def cropsample(img):
    c = randint(0,4)
    if c==0:
        return resize(img[:104,:104], (img_size_target, img_size_target), mode='constant', preserve_range=True)
    elif c==1:
        return resize(img[24:,:104], (img_size_target, img_size_target), mode='constant', preserve_range=True)
    elif c==2:
        return resize(img[:104,24:], (img_size_target, img_size_target), mode='constant', preserve_range=True)
    else:
        return resize(img[24:,24:], (img_size_target, img_size_target), mode='constant', preserve_range=True)

In [ ]:
x_train = np.append(x_train, [np.fliplr(x) for x in x_train], axis=0)
y_train = np.append(y_train, [np.fliplr(x) for x in y_train], axis=0)
print(len(x_train), len(y_train))

In [ ]:
random.seed(1007)
x_train = np.append(x_train, [cropsample(x) for x in tqdm_notebook(x_train)], axis=0)
random.seed(1007)
y_train = np.append(y_train, [cropsample(x) for x in tqdm_notebook(y_train)], axis=0)
print(len(x_train), len(y_train))

In [ ]:
fig, axs = plt.subplots(2, 10, figsize=(15,3))
for i in range(10):
    axs[0][i].imshow(x_train[i].squeeze(), cmap="Greys")
    axs[0][i].imshow(y_train[i].squeeze(), cmap="Greens", alpha=0.3)
    axs[1][i].imshow(x_train[int(len(x_train)/2 + i)].squeeze(), cmap="Greys")
    axs[1][i].imshow(y_train[int(len(y_train)/2 + i)].squeeze(), cmap="Greens", alpha=0.3)
fig.suptitle("Top row: original images, bottom row: augmented images")

# Training

In [ ]:
import math
from keras.callbacks import LearningRateScheduler
def step_decay(epoch):
    initial_lrate = 0.0012
    drop = 0.5
    epochs_drop = 8.0
    lrate = initial_lrate * math.pow(drop,math.floor((1+epoch)/epochs_drop))
    print("lrate reduced to", lrate)
    return lrate

In [ ]:
early_stopping = EarlyStopping(patience=20, verbose=1)
model_checkpoint = ModelCheckpoint("./keras4.model", save_best_only=True, verbose=1)
lrate = LearningRateScheduler(step_decay)

epochs = 60
batch_size = 16

history = model.fit(x_train, y_train,
                    validation_data=[x_valid, y_valid], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint, lrate],shuffle=True)

In [ ]:
fig, (ax_loss, ax_acc) = plt.subplots(1, 2, figsize=(15,5))
ax_loss.plot(history.epoch, history.history["loss"], label="Train loss")
ax_loss.plot(history.epoch, history.history["val_loss"], label="Validation loss")
ax_acc.plot(history.epoch, history.history["my_iou_metric"], label="Train accuracy")
ax_acc.plot(history.epoch, history.history["val_my_iou_metric"], label="Validation accuracy")

In [ ]:
save_model(model, "./endkeras4.model")

In [ ]:
# end